# 导入工具包

In [1]:
# !pip list
# tensorflow                         2.2.0 
# bert4keras                         0.9.3
# Keras                              2.3.1 
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import lightgbm as lgb

# 读取数据

In [2]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
results = pd.read_csv("../data/results.csv")

# 查看训练数据

In [3]:
train.head()

,id,level_1,level_2,level_3,level_4,content,label
0,0,工业/危化品类（现场）—2016版,（二）电气安全,6、移动用电产品、电动工具及照明,1、移动使用的用电产品和I类电动工具的绝缘线，必须采用三芯(单相)或四芯(三相)多股铜芯橡套软线。,"使用移动手动电动工具,外接线绝缘皮破损,应停止使用.",0
1,1,工业/危化品类（现场）—2016版,（一）消防检查,1、防火巡查,3、消防设施、器材和消防安全标志是否在位、完整；,一般,1
2,2,工业/危化品类（现场）—2016版,（一）消防检查,2、防火检查,6、重点工种人员以及其他员工消防知识的掌握情况；,消防知识要加强,0
3,3,工业/危化品类（现场）—2016版,（一）消防检查,1、防火巡查,3、消防设施、器材和消防安全标志是否在位、完整；,消防通道有货物摆放 清理不及时,0
4,4,工业/危化品类（现场）—2016版,（一）消防检查,1、防火巡查,4、常闭式防火门是否处于关闭状态，防火卷帘下是否堆放物品影响使用；,防火门打开状态,0


# 查看训练数据条数

In [4]:
train.shape

(12000, 7)

# 查看测试数据

In [5]:
test.head()

,id,level_1,level_2,level_3,level_4,content
0,0,交通运输类（现场）—2016版,（一）消防安全,2、防火检查,2、安全疏散通道、疏散指示标志、应急照明和安全出口情况。,RB1洗地机占用堵塞安全通道
1,1,工业/危化品类（选项）—2016版,（二）仓库,1、一般要求,1、库房内储存物品应分类、分堆、限额存放。,未分类堆放
2,2,工业/危化品类（现场）—2016版,（一）消防检查,1、防火巡查,3、消防设施、器材和消防安全标志是否在位、完整；,消防设施、器材和消防安全标志是否在位、完整
3,3,商贸服务教文卫类（现场）—2016版,（二）电气安全,3、电气线路及电源插头插座,3、电源插座、电源插头应按规定正确接线。,插座随意放在电器旁边
4,4,商贸服务教文卫类（现场）—2016版,（一）消防检查,1、防火巡查,6、其他消防安全情况。,检查中发现一瓶灭火器过期


# 训练集去重

In [7]:
train = train.drop_duplicates(['level_1', 'level_2', 'level_3', 'level_4', 'content', 'label'])

# 查看去重训练数据条数

In [8]:
train.shape

(10619, 7)

In [9]:
# !pip install textvec

# 构建特征

In [10]:
train['text'] = ( train['content']
              ).map(lambda x:' '.join(list(str(x))))
test['text'] = (test['content']
             ).map(lambda x:' '.join(list(str(x))))

In [11]:
' '.join(list(str('未分类堆放')))

'未 分 类 堆 放'

In [12]:
'未 分类 堆放' 分词

SyntaxError: invalid syntax (<ipython-input-12-d541ec9c3f78>, line 1)

In [13]:
list('未分类堆放')

['未', '分', '类', '堆', '放']

In [14]:
ngram
1gram
'未', '分', '类', '堆', '放'

2gram 
'未分', '分类', '类堆', '堆放'

3gram
'未分类', '分类堆', '类堆放'

SyntaxError: invalid syntax (<ipython-input-14-4bd0c12b99e7>, line 2)

In [15]:
ngram_range=(1, 1) 
1gram
'未', '分', '类', '堆', '放'

SyntaxError: invalid syntax (<ipython-input-15-bd0a77512f80>, line 2)

In [ ]:
ngram_range=(1, 3) 
1gram
'未', '分', '类', '堆', '放'

2gram 
'未分', '分类', '类堆', '堆放'

3gram
'未分类', '分类堆', '类堆放'

In [16]:
vectorizer = CountVectorizer(analyzer='char', ngram_range=(1, 3), stop_words=[])

In [17]:
train_X = vectorizer.fit_transform(train['text']).toarray()
test_X = vectorizer.transform(test['text']).toarray()

In [18]:
train_X

array([[ 0, 25,  0, ...,  0,  0,  0],
       [ 0,  1,  0, ...,  0,  0,  0],
       [ 0,  6,  0, ...,  0,  0,  0],
       ...,
       [ 0, 61,  0, ...,  0,  0,  0],
       [ 0, 15,  0, ...,  0,  0,  0],
       [ 0, 16,  0, ...,  0,  0,  0]])

In [19]:
train_y = train['label'].astype(int).values

In [20]:
vectorizer.get_feature_names()[10:]

[' (',
 ' ( ',
 ' )',
 ' ) ',
 ' +',
 ' + ',
 ' ,',
 ' , ',
 ' -',
 ' - ',
 ' .',
 ' . ',
 ' /',
 ' / ',
 ' 0',
 ' 0 ',
 ' 1',
 ' 1 ',
 ' 2',
 ' 2 ',
 ' 3',
 ' 3 ',
 ' 4',
 ' 4 ',
 ' 5',
 ' 5 ',
 ' 6',
 ' 6 ',
 ' 7',
 ' 7 ',
 ' 8',
 ' 8 ',
 ' 9',
 ' 9 ',
 ' :',
 ' : ',
 ' ;',
 ' ; ',
 ' \\',
 ' \\ ',
 ' a',
 ' a ',
 ' b',
 ' b ',
 ' c',
 ' c ',
 ' d',
 ' d ',
 ' e',
 ' e ',
 ' f',
 ' f ',
 ' g',
 ' g ',
 ' h',
 ' h ',
 ' i',
 ' i ',
 ' k',
 ' k ',
 ' l',
 ' l ',
 ' m',
 ' m ',
 ' n',
 ' n ',
 ' o',
 ' o ',
 ' p',
 ' p ',
 ' q',
 ' q ',
 ' r',
 ' r ',
 ' s',
 ' s ',
 ' t',
 ' t ',
 ' u',
 ' u ',
 ' v',
 ' v ',
 ' w',
 ' w ',
 ' x',
 ' x ',
 ' z',
 ' z ',
 ' ~',
 ' ~ ',
 ' ·',
 ' · ',
 ' —',
 ' — ',
 ' ‘',
 ' ‘ ',
 ' ’',
 ' ’ ',
 ' “',
 ' “ ',
 ' ”',
 ' ” ',
 ' …',
 ' … ',
 ' √',
 ' ≤',
 ' ≤ ',
 ' ≥',
 ' ≥ ',
 ' ②',
 ' ② ',
 ' 、',
 ' 、 ',
 ' 。',
 ' 。 ',
 ' 《',
 ' 《 ',
 ' 》',
 ' 》 ',
 ' 】',
 ' 】 ',
 ' ㎜',
 ' 一',
 ' 一 ',
 ' 七',
 ' 七 ',
 ' 万',
 ' 万 ',
 ' 三',
 ' 三 ',
 ' 上',
 ' 上 ',
 ' 下',
 '

In [21]:
train_X.shape

(10619, 30215)

In [22]:
test_X.shape

(18000, 30215)

# 五折交叉验证

In [23]:
params = {
        'task':'train',
        'boosting_type':'gbdt',
        'num_leaves': 31,
        'objective': 'binary', 
        'learning_rate': 0.05, 
        'bagging_freq': 2, 
        'max_bin':256,
        'num_threads': 32,
#         'metric':['binary_logloss','binary_error']
    } 

In [24]:
skf = StratifiedKFold(n_splits=5)

In [25]:
for index,(train_index, test_index) in enumerate(skf.split(train_X, train_y)):
    X_train, X_test = train_X[train_index], train_X[test_index]
    y_train, y_test = train_y[train_index], train_y[test_index]
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    
    gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1000,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)
    y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
    pred = gbm.predict(test_X, num_iteration=gbm.best_iteration)
    if index == 0:
        pred_y_check, true_y_check = list(y_pred), list(y_test)
        pred_out=pred
    else:
        pred_y_check += list(y_pred)
        true_y_check += list(y_test)
        pred_out += pred
        

[LightGBM] [Info] Number of positive: 611, number of negative: 7884
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.179312 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8694
[LightGBM] [Info] Number of data points in the train set: 8495, number of used features: 2796
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.071925 -> initscore=-2.557494
[LightGBM] [Info] Start training from score -2.557494
[1]	valid_0's binary_logloss: 0.23485
Training until validation scores don't improve for 10 rounds
[2]	valid_0's binary_logloss: 0.221137
[3]	valid_0's binary_logloss: 0.21116
[4]	valid_0's binary_logloss: 0.203163
[5]	valid_0's binary_logloss: 0.196411
[6]	valid_0's binary_logloss: 0.190586
[7]	valid_0's binary_logloss: 0.184789
[8]	valid_0's binary_logloss: 0.180014
[9]	valid_0's binary_logloss: 0.174707
[10]	valid_0's binary_logloss: 0.170154
[11]	valid_0's binary_logloss: 0.165865
[12]	valid_0's 

[28]	valid_0's binary_logloss: 0.121874
[29]	valid_0's binary_logloss: 0.120072
[30]	valid_0's binary_logloss: 0.118173
[31]	valid_0's binary_logloss: 0.116883
[32]	valid_0's binary_logloss: 0.115317
[33]	valid_0's binary_logloss: 0.113109
[34]	valid_0's binary_logloss: 0.111416
[35]	valid_0's binary_logloss: 0.109836
[36]	valid_0's binary_logloss: 0.108434
[37]	valid_0's binary_logloss: 0.106888
[38]	valid_0's binary_logloss: 0.105243
[39]	valid_0's binary_logloss: 0.104165
[40]	valid_0's binary_logloss: 0.103338
[41]	valid_0's binary_logloss: 0.102531
[42]	valid_0's binary_logloss: 0.101384
[43]	valid_0's binary_logloss: 0.100344
[44]	valid_0's binary_logloss: 0.0993751
[45]	valid_0's binary_logloss: 0.0981774
[46]	valid_0's binary_logloss: 0.0972578
[47]	valid_0's binary_logloss: 0.0966034
[48]	valid_0's binary_logloss: 0.095703
[49]	valid_0's binary_logloss: 0.0950267
[50]	valid_0's binary_logloss: 0.0941678
[51]	valid_0's binary_logloss: 0.0934855
[52]	valid_0's binary_logloss: 0.

[33]	valid_0's binary_logloss: 0.114918
[34]	valid_0's binary_logloss: 0.113473
[35]	valid_0's binary_logloss: 0.111845
[36]	valid_0's binary_logloss: 0.111156
[37]	valid_0's binary_logloss: 0.109761
[38]	valid_0's binary_logloss: 0.109022
[39]	valid_0's binary_logloss: 0.107751
[40]	valid_0's binary_logloss: 0.106906
[41]	valid_0's binary_logloss: 0.106209
[42]	valid_0's binary_logloss: 0.10498
[43]	valid_0's binary_logloss: 0.103495
[44]	valid_0's binary_logloss: 0.102296
[45]	valid_0's binary_logloss: 0.100981
[46]	valid_0's binary_logloss: 0.0997087
[47]	valid_0's binary_logloss: 0.0986129
[48]	valid_0's binary_logloss: 0.0974807
[49]	valid_0's binary_logloss: 0.0964358
[50]	valid_0's binary_logloss: 0.0953697
[51]	valid_0's binary_logloss: 0.0946628
[52]	valid_0's binary_logloss: 0.0938728
[53]	valid_0's binary_logloss: 0.0926817
[54]	valid_0's binary_logloss: 0.0915917
[55]	valid_0's binary_logloss: 0.0905507
[56]	valid_0's binary_logloss: 0.089918
[57]	valid_0's binary_logloss: 

[64]	valid_0's binary_logloss: 0.0836885
[65]	valid_0's binary_logloss: 0.0832215
[66]	valid_0's binary_logloss: 0.0829603
[67]	valid_0's binary_logloss: 0.0826434
[68]	valid_0's binary_logloss: 0.0823107
[69]	valid_0's binary_logloss: 0.0820157
[70]	valid_0's binary_logloss: 0.0817244
[71]	valid_0's binary_logloss: 0.0812204
[72]	valid_0's binary_logloss: 0.0809883
[73]	valid_0's binary_logloss: 0.0807915
[74]	valid_0's binary_logloss: 0.0805353
[75]	valid_0's binary_logloss: 0.0801101
[76]	valid_0's binary_logloss: 0.079753
[77]	valid_0's binary_logloss: 0.079301
[78]	valid_0's binary_logloss: 0.0790703
[79]	valid_0's binary_logloss: 0.0786601
[80]	valid_0's binary_logloss: 0.0783272
[81]	valid_0's binary_logloss: 0.0781536
[82]	valid_0's binary_logloss: 0.078043
[83]	valid_0's binary_logloss: 0.0778697
[84]	valid_0's binary_logloss: 0.0776138
[85]	valid_0's binary_logloss: 0.0771578
[86]	valid_0's binary_logloss: 0.0771489
[87]	valid_0's binary_logloss: 0.0769702
[88]	valid_0's bina

[104]	valid_0's binary_logloss: 0.0734473
[105]	valid_0's binary_logloss: 0.0733524
[106]	valid_0's binary_logloss: 0.0732308
[107]	valid_0's binary_logloss: 0.0731133
[108]	valid_0's binary_logloss: 0.0729332
[109]	valid_0's binary_logloss: 0.0726761
[110]	valid_0's binary_logloss: 0.0723932
[111]	valid_0's binary_logloss: 0.0723208
[112]	valid_0's binary_logloss: 0.0722538
[113]	valid_0's binary_logloss: 0.0722971
[114]	valid_0's binary_logloss: 0.0720457
[115]	valid_0's binary_logloss: 0.0719214
[116]	valid_0's binary_logloss: 0.0717892
[117]	valid_0's binary_logloss: 0.0715512
[118]	valid_0's binary_logloss: 0.0715824
[119]	valid_0's binary_logloss: 0.0714408
[120]	valid_0's binary_logloss: 0.0711992
[121]	valid_0's binary_logloss: 0.0712449
[122]	valid_0's binary_logloss: 0.0711579
[123]	valid_0's binary_logloss: 0.0710705
[124]	valid_0's binary_logloss: 0.0708958
[125]	valid_0's binary_logloss: 0.0706415
[126]	valid_0's binary_logloss: 0.0704612
[127]	valid_0's binary_logloss: 0.

# 线下验证

In [26]:
for i in range(10):
    pred = [int(x) for x in np.where(np.array(pred_y_check) >= i/10.0,1,0)]
    scores = f1_score(true_y_check,pred)
    print(i, scores)

0 0.13423526311165773
1 0.7626208378088076
2 0.8177991581479254
3 0.813624678663239
4 0.8190988567585744
5 0.8055944055944056
6 0.7841989758595465
7 0.7528691660290742
8 0.7035830618892508
9 0.6027149321266968


In [27]:
for i in range(10):
    pred = [int(x) for x in np.where(np.array(pred_y_check) >= i/10.0,1,0)]
    scores = f1_score(true_y_check,pred)
    print(i/10.0, scores)

0.0 0.13423526311165773
0.1 0.7626208378088076
0.2 0.8177991581479254
0.3 0.813624678663239
0.4 0.8190988567585744
0.5 0.8055944055944056
0.6 0.7841989758595465
0.7 0.7528691660290742
0.8 0.7035830618892508
0.9 0.6027149321266968


In [28]:
train_y.shape

(10619,)

In [29]:
np.sum(train_y)

764